<a href="https://colab.research.google.com/github/konevenkatesh/SupplementOntologyData/blob/master/test/iprok_resource.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install owlready2
!pip install rdflib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 46.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for owlready2: filename=owlready2-0.47-cp311-cp311-linux_x86_64.whl size=24521243 sha256=a6fc492bd8fa90cb5386496f3c7a104789e23a3813db5091631984fdb087ab00
  Stored in directory: /root/.cache/pip/wheels/25/9a/a3/fb1ac6339caa859c8bb18d685736168b0b51d851af13d81d52
Successfully built owlready2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 8.6 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
from owlready2 import *
from datetime import date, datetime, timedelta
from rdflib import Graph

In [3]:
set_log_level(9)

In [4]:
# Define the IproK Ontology IRI and get/create the ontology
resource_iri = "https://w3id.org/iprok/resource#"
resource_iprok = get_ontology(resource_iri)

* Owlready2 * Creating new ontology resource <https://w3id.org/iprok/resource#>.
* Owlready2 * ADD TRIPLE https://w3id.org/iprok/resource http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Ontology


In [5]:
# Improt base IproK ontology
iprok = get_ontology("iprok.owl").load()
resource_iprok.imported_ontologies.append(iprok)

* Owlready2 * Creating new ontology iprok <iprok.owl#>.
* Owlready2 * ADD TRIPLE iprok.owl http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Ontology
* Owlready2 *     ...loading ontology iprok from iprok.owl...
* Owlready2 *     ...6 properties found: constraintType, constraintDate, hasGlobalId, hasId, hasName, hasDescription
* Owlready2 * ADD TRIPLE https://w3id.org/iprok/resource http://www.w3.org/2002/07/owl#imports https://w3id.org/iprok


In [6]:

with resource_iprok:

    # Core Classes
    class Resource(iprok.IproKThing):
        label = ["Resource"]
        comment = ["An asset, item, or person required for or consumed by project tasks."]
        # namespace = resource_iprok # Explicitly set namespace if needed
        pass

    class ResourceType(iprok.IproKThing):
        label = ["Resource Type"]
        comment = ["The classification of a resource (e.g., labor, equipment, material)."]
        # namespace = resource_iprok
        pass

    class LaborResource(ResourceType):
        label = ["Labor Resource"]
        comment = ["A type of resource representing human effort, skills, or services."]
        # namespace = resource_iprok
        pass

    class EquipmentResource(ResourceType):
        label = ["Equipment Resource"]
        comment = ["A type of resource representing machinery, tools, or other equipment."]
        # namespace = resource_iprok
        pass

    class MaterialResource(ResourceType):
        label = ["Material Resource"]
        comment = ["A type of resource representing consumable supplies or raw materials."]
        # namespace = resource_iprok
        pass

    class UserDefinedResource(ResourceType):
        label = ["User Defined Resource"]
        comment = ["A flexible type for resources not covered by standard classifications like labor, equipment, or material."]
        # namespace = resource_iprok
        pass

    # Axiom: Resource types are disjoint
    AllDisjoint([LaborResource, EquipmentResource, MaterialResource, UserDefinedResource])

    class TrackingTag(iprok.IproKThing):
        label = ["Tracking Tag"]
        comment = ["An identifier or device (e.g., RFID, barcode) used for monitoring and tracking a resource."]
        # namespace = resource_iprok
        pass

    class Units(iprok.IproKThing): # Unit of Measure
        label = ["Units"]
        comment = ["The unit of measure for quantifying a resource's amount, capacity, or rate (e.g., hours, kg, items)."]
        # namespace = resource_iprok
        pass

    class ResourceConstraint(iprok.Constraint):
        label = ["Resource Constraint"]
        comment = ["A specific limitation or condition affecting the use, availability, or allocation of a resource."]
        # namespace = resource_iprok
        pass

    # Resource Module - Data Properties
    # Reusing hasId, hasName, hasGlobalId, hasDescription defined in 'iprok'
    # These should already be annotated in their original 'iprok' definition.
    iprok.hasId.domain.append(Resource)
    iprok.hasName.domain.append(Resource)
    iprok.hasGlobalId.domain.append(Resource)
    iprok.hasDescription.domain.append(Resource)
    # Consider also extending domains for ResourceType, TrackingTag, Units, ResourceConstraint if they need these common properties.
    # e.g., iprok.hasName.domain.extend([ResourceType, Units, TrackingTag])


    class baseRate(DataProperty):
        # namespace = resource_iprok
        label = ["base Rate"]
        comment = ["The standard cost rate for a resource, typically per unit of time or quantity."]
        domain = [Resource]
        range = [float]

    class currency(DataProperty, FunctionalProperty):
        # namespace = resource_iprok
        label = ["currency"]
        comment = ["The currency code (e.g., USD, EUR) for monetary values related to the resource, like its base rate."]
        domain = [Resource] # This might also apply to CostItem in another module
        range = [str]

    class effectiveDateForRate(DataProperty, FunctionalProperty):
        # namespace = resource_iprok
        label = ["effective Date For Rate"]
        comment = ["The date from which the specified base rate for a resource becomes effective."]
        domain = [Resource]
        range = [date]

    class maxUnits(DataProperty, FunctionalProperty):
        # namespace = resource_iprok
        label = ["max Units"]
        comment = ["The maximum available units of a resource, often per a specific time period (e.g., maximum work hours per day for a labor resource)."]
        domain = [Resource]
        range = [float]

    class budgetedUnits(DataProperty):
        # namespace = resource_iprok
        label = ["budgeted Units"]
        comment = ["The planned or budgeted quantity of resource units. (Note: Often better placed on a ResourceAllocation linking resource to task)."]
        domain = [Resource] # Or ResourceAllocation
        range = [float]

    class actualUnits(DataProperty):
        # namespace = resource_iprok
        label = ["actual Units"]
        comment = ["The actual quantity of resource units consumed or utilized. (Note: Often better placed on a ResourceAllocation)."]
        domain = [Resource] # Or ResourceAllocation
        range = [float]

    class remainingUnits(DataProperty):
        # namespace = resource_iprok
        label = ["remaining Units"]
        comment = ["The estimated quantity of resource units still required or available. (Note: Often better placed on a ResourceAllocation)."]
        domain = [Resource] # Or ResourceAllocation
        range = [float]

    class atCompleteUnits(DataProperty):
        # namespace = resource_iprok
        label = ["at Complete Units"]
        comment = ["The total estimated quantity of resource units at the completion of its assignment. (Note: Often better placed on a ResourceAllocation)."]
        domain = [Resource] # Or ResourceAllocation
        range = [float]

    class isPrimaryResource(DataProperty, FunctionalProperty):
        # namespace = resource_iprok
        label = ["is Primary Resource"]
        comment = ["A boolean flag indicating if this resource is considered the primary one for an assignment or task."]
        domain = [Resource] # Or ResourceAllocation
        range = [bool]

    # ResourceConstraint data properties (like constraintType, constraintDate) are inherited from iprok.Constraint if defined there.

    # Resource Module - Object Properties
    class hasResourceType(ObjectProperty, FunctionalProperty):
        # namespace = resource_iprok
        label = ["has Resource Type"]
        comment = ["Links a resource to its specific classification or type (e.g., LaborResource, EquipmentResource)."]
        domain = [Resource]
        range = [ResourceType]

    class hasTrackingTag(ObjectProperty):
        # namespace = resource_iprok
        label = ["has Tracking Tag"]
        comment = ["Links a resource to one or more tracking tags used for its identification and monitoring."]
        domain = [Resource]
        range = [TrackingTag]

    class measuredIn(ObjectProperty, FunctionalProperty):
        # namespace = resource_iprok
        label = ["measured In"]
        comment = ["Links a resource to the unit of measure used for its quantification."]
        domain = [Resource]
        range = [Units]

    class hasResourceConstraints(ObjectProperty):
        # namespace = resource_iprok
        label = ["has Resource Constraints"]
        comment = ["Links a resource to specific constraints that apply to its availability or usage."]
        domain = [Resource]
        range = [ResourceConstraint]

* Owlready2 * ADD TRIPLE https://w3id.org/iprok/resource#Resource http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Class
* Owlready2 * ADD TRIPLE https://w3id.org/iprok/resource#Resource http://www.w3.org/2000/01/rdf-schema#subClassOf https://w3id.org/iprok#IproKThing
* Owlready2 * ADD TRIPLE https://w3id.org/iprok/resource#Resource http://www.w3.org/2000/01/rdf-schema#label Resource http://www.w3.org/2001/XMLSchema#string
* Owlready2 * ADD TRIPLE https://w3id.org/iprok/resource#Resource http://www.w3.org/2000/01/rdf-schema#comment An asset, item, or person required for or consumed by project tasks. http://www.w3.org/2001/XMLSchema#string
* Owlready2 * ADD TRIPLE https://w3id.org/iprok/resource#ResourceType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Class
* Owlready2 * ADD TRIPLE https://w3id.org/iprok/resource#ResourceType http://www.w3.org/2000/01/rdf-schema#subClassOf https://w3id.org/iprok#IproKThing
* Owlready2 * ADD 

In [7]:
resource_iprok.save(file="iprok_resource.owl", format="rdfxml")
g = Graph()
g.parse("iprok_resource.owl", format="xml")
g.serialize(destination="iprok_resource.ttl", format="turtle")


* Owlready2 * Saving ontology resource to iprok_resource.owl...


<Graph identifier=N321e00284e8e4631b7774484d178591c (<class 'rdflib.graph.Graph'>)>